#Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [5]:
import pandas as pd
import numpy as np

Dataset Loading

In [6]:
ratings = pd.read_csv('/content/cleaned_ratings.csv')
anime_df = pd.read_csv('/content/cleaned_anime.csv')

#Interaction Matrix

In [7]:
# Drop duplicate entries
ratings= ratings.drop_duplicates(subset=['user_id', 'anime_id'])

# Then proceed with creating the interaction matrix
interaction_matrix = (
    ratings.set_index(['user_id', 'anime_id'])['rating']
    .unstack(fill_value=0)
)

print(interaction_matrix)


anime_id  NaN      1.0      5.0      6.0      7.0      8.0      15.0     \
user_id                                                                   
1             0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2             0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3             0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4             0.0      0.0      0.0     -1.0      0.0      0.0      0.0   
5             0.0      0.0      0.0      8.0      0.0      0.0      6.0   
...           ...      ...      ...      ...      ...      ...      ...   
30860         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
30861         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
30862         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
30863         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
30864         NaN     10.0     -1.0      0.0      0.0      0.0      0.0   

anime_id  16.0     17.0 

#Finding Pearson Correlation For Users

In [8]:
pearson_correlation = np.corrcoef(interaction_matrix.T)


#Ratings Prediction Code using Pearson Correlation

In [9]:
def prediction(ratings, scores, interaction_matrix, user, item):
    result = 0
    denom = 0

    # mean rating of the user
    user_mean_rating = interaction_matrix.loc[user].mean()

    for i in range(len(scores)):
        similar_user = ratings[i]
        similarity_score = scores[i]

        if similar_user in interaction_matrix.index and item in interaction_matrix.columns:
            user_rating = interaction_matrix.loc[similar_user, item]
            similar_user_mean_rating = interaction_matrix.loc[similar_user].mean()
            mean_centered_rating = user_rating - similar_user_mean_rating
            result += similarity_score * mean_centered_rating
            denom += abs(similarity_score)
    if denom == 0:
        return user_mean_rating

    result /= denom
    result += user_mean_rating

    return result


#Finding Similar User Using Pearson Correlation

In [10]:
# find k most similar users
def find_similar_users(k, user, pearson_correlation, interaction_matrix):
    k += 1
    user_index = interaction_matrix.index.get_loc(user)
    similarity_scores = pearson_correlation[user_index]
    similar_users_indices = np.argsort(similarity_scores)[::-1][:k]
    similar_users_indices = similar_users_indices[similar_users_indices != user_index]
    similar_users = interaction_matrix.index[similar_users_indices]
    return list(similar_users), similarity_scores[similar_users_indices]


#Inference Code

In [11]:
user_id = 3
item_id = 226
k = 5
similar_users, scores = find_similar_users(k, user_id, pearson_correlation, interaction_matrix)
predicted_rating = prediction(similar_users, scores, interaction_matrix, user_id, item_id)
print("Predicted Rating:", predicted_rating)


Predicted Rating: nan


In [12]:
def recommend_anime(user_id, interaction_matrix, pearson_correlation, k, anime_df, top_n=5):
    all_anime_ids = interaction_matrix.columns
    rated_anime_ids = interaction_matrix.loc[user_id][interaction_matrix.loc[user_id] > 0].index
    anime_to_predict = [anime_id for anime_id in all_anime_ids if anime_id not in rated_anime_ids]
    predictions = []
    for anime_id in anime_to_predict:
        similar_users, scores = find_similar_users(k, user_id, pearson_correlation, interaction_matrix)
        predicted_rating = prediction(similar_users, scores, interaction_matrix, user_id, anime_id)
        predictions.append((anime_id, predicted_rating))
    predictions.sort(key=lambda x: x[1], reverse=True)

    top_anime_ids = [anime_id for anime_id, _ in predictions[:top_n]]

    recommended_anime = anime_df[anime_df['anime_id'].isin(top_anime_ids)]

    return recommended_anime

user_id = 3
k = 5
top_n = 5

recommended_anime = recommend_anime(user_id, interaction_matrix, pearson_correlation, k, anime_df, top_n)
print("Top Recommended Anime for User", user_id)
print(recommended_anime[['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members']])

Top Recommended Anime for User 3
      anime_id                             name  \
22           1                     Cowboy Bebop   
152          5  Cowboy Bebop: Tengoku no Tobira   
214          6                           Trigun   
2094         7               Witch Hunter Robin   

                                                  genre   type  episodes  \
22      Action, Adventure, Comedy, Drama, Sci-Fi, Space     TV      26.0   
152               Action, Drama, Mystery, Sci-Fi, Space  Movie       1.0   
214                              Action, Comedy, Sci-Fi     TV      26.0   
2094  Action, Drama, Magic, Mystery, Police, Superna...     TV      26.0   

      rating  members  
22      8.82   486824  
152     8.40   137636  
214     8.32   283069  
2094    7.36    64905  


#Calculate Loss Between Actual And predicted Values

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def recommend_anime_with_metrics(user_id, interaction_matrix, pearson_correlation, k, anime_df, ratings_df, top_n=5):
    """
    Recommend top_n anime for a given user and calculate MSE and MAE.

    Parameters:
        user_id: ID of the user.
        interaction_matrix: User-item interaction matrix.
        pearson_correlation: Precomputed Pearson correlation matrix.
        k: Number of similar users to consider.
        anime_df: DataFrame containing anime details.
        ratings_df: DataFrame containing user ratings.
        top_n: Number of anime recommendations to return (default: 5).

    Returns:
        recommended_anime: DataFrame of recommended anime.
        mse: Mean Squared Error of predictions.
        mae: Mean Absolute Error of predictions.
    """
    all_anime_ids = interaction_matrix.columns
    rated_anime_ids = interaction_matrix.loc[user_id][interaction_matrix.loc[user_id] > 0].index
    anime_to_predict = [anime_id for anime_id in all_anime_ids if anime_id not in rated_anime_ids]

    predictions = []
    actual_ratings = []

    for anime_id in anime_to_predict:
        # Find similar users and their scores
        similar_users, scores = find_similar_users(k, user_id, pearson_correlation, interaction_matrix)

        # Predict the rating for the current anime
        predicted_rating = prediction(similar_users, scores, interaction_matrix, user_id, anime_id)
        predictions.append((anime_id, predicted_rating))

        # Retrieve the actual rating if it exists
        actual_rating = ratings_df[(ratings_df['user_id'] == user_id) & (ratings_df['anime_id'] == anime_id)]
        if not actual_rating.empty:
            actual_ratings.append(actual_rating['rating'].values[0])

    # Sort predictions by predicted rating
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Select the top-n recommended anime
    top_anime_ids = [anime_id for anime_id, _ in predictions[:top_n]]
    recommended_anime = anime_df[anime_df['anime_id'].isin(top_anime_ids)]

    # Filter actual ratings and predicted ratings for MSE/MAE calculation
    top_actual_ratings = []
    top_predicted_ratings = []

    for anime_id, predicted_rating in predictions[:top_n]:
        actual_rating = ratings_df[(ratings_df['user_id'] == user_id) & (ratings_df['anime_id'] == anime_id)]
        if not actual_rating.empty:
            top_actual_ratings.append(actual_rating['rating'].values[0])
            top_predicted_ratings.append(predicted_rating)

    # Calculate MSE and MAE
    if top_actual_ratings:
        mse = mean_squared_error(top_actual_ratings, top_predicted_ratings)
        mae = mean_absolute_error(top_actual_ratings, top_predicted_ratings)
    else:
        mse = mae = None  # No actual ratings available for comparison

    return recommended_anime, mse, mae


# Example Usage
user_id = 3
k = 5
top_n = 5

recommended_anime, mse, mae = recommend_anime_with_metrics(user_id, interaction_matrix, pearson_correlation, k, anime_df, ratings, top_n)

print("Top Recommended Anime for User", user_id)
print(recommended_anime[['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members']])

if mse is not None and mae is not None:
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
else:
    print("No actual ratings available to calculate MSE and MAE.")


Top Recommended Anime for User 3
      anime_id                             name  \
22           1                     Cowboy Bebop   
152          5  Cowboy Bebop: Tengoku no Tobira   
214          6                           Trigun   
2094         7               Witch Hunter Robin   

                                                  genre   type  episodes  \
22      Action, Adventure, Comedy, Drama, Sci-Fi, Space     TV      26.0   
152               Action, Drama, Mystery, Sci-Fi, Space  Movie       1.0   
214                              Action, Comedy, Sci-Fi     TV      26.0   
2094  Action, Drama, Magic, Mystery, Police, Superna...     TV      26.0   

      rating  members  
22      8.82   486824  
152     8.40   137636  
214     8.32   283069  
2094    7.36    64905  
No actual ratings available to calculate MSE and MAE.


# Importing Libraries

In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam

Loading Data

In [16]:
anime_df = pd.read_csv('/content/cleaned_anime.csv')
ratings_df = pd.read_csv('/content/cleaned_ratings.csv')

Cleaning Null and Unrated Animes

In [ ]:
import tensorflow as tf

print("Is GPU available:", tf.test.is_gpu_available())
print("Available devices:", tf.config.list_physical_devices())


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Is GPU available: False
Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [17]:
ratings_df = ratings_df[ratings_df.rating.notna()]
ratings_df = ratings_df[ratings_df['rating'] > 0]

#Preprocessing
Encoding User ids and animes id and also normalizing ratings

In [18]:
# Encode user_id and anime_id
user_encoder = LabelEncoder()
anime_encoder = LabelEncoder()

ratings_df['user_id'] = user_encoder.fit_transform(ratings_df['user_id'])
ratings_df['anime_id'] = anime_encoder.fit_transform(ratings_df['anime_id'])

num_users = ratings_df['user_id'].nunique()
num_animes = ratings_df['anime_id'].nunique()

# Normalize ratings
ratings_df['rating'] = ratings_df['rating'] / 10.0  # Scale ratings to [0, 1]

# Train-test split
train, test = train_test_split(ratings_df, test_size=0.1, random_state=42)

# Neural network inputs
user_input = Input(shape=(1,), name='user_input')
anime_input = Input(shape=(1,), name='anime_input')

# Embedding layers
user_embedding = Embedding(input_dim=num_users, output_dim=50, name='user_embedding')(user_input)
anime_embedding = Embedding(input_dim=num_animes, output_dim=50, name='anime_embedding')(anime_input)

# Flatten embeddings
user_flat = Flatten()(user_embedding)
anime_flat = Flatten()(anime_embedding)

# Concatenate embeddings
concat = Concatenate()([user_flat, anime_flat])

#Nueral Network Architechture

In [ ]:
# Encode user_id and anime_id
user_encoder = LabelEncoder()
anime_encoder = LabelEncoder()

ratings_df['user_id'] = user_encoder.fit_transform(ratings_df['user_id'])
ratings_df['anime_id'] = anime_encoder.fit_transform(ratings_df['anime_id'])

num_users = ratings_df['user_id'].nunique()
num_animes = ratings_df['anime_id'].nunique()

# Normalize ratings
ratings_df['rating'] = ratings_df['rating'] / 10.0  # Scale ratings to [0, 1]

# Train-test split
train, test = train_test_split(ratings_df, test_size=0.1, random_state=42)

# Neural network inputs
user_input = Input(shape=(1,), name='user_input')
anime_input = Input(shape=(1,), name='anime_input')

# Embedding layers
user_embedding = Embedding(input_dim=num_users, output_dim=50, name='user_embedding')(user_input)
anime_embedding = Embedding(input_dim=num_animes, output_dim=50, name='anime_embedding')(anime_input)

# Flatten embeddings
user_flat = Flatten()(user_embedding)
anime_flat = Flatten()(anime_embedding)

# Concatenate embeddings
concat = Concatenate()([user_flat, anime_flat])

# Fully connected layers
dense1 = Dense(128, activation='relu')(concat)
dropout1 = Dropout(0.3)(dense1)
dense2 = Dense(64, activation='relu')(dense1)
dropout2 = Dropout(0.2)(dense2)
output = Dense(1, activation='sigmoid')(dense2)  # Sigmoid to predict normalized ratings

# Define model
model = Model(inputs=[user_input, anime_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
with tf.device('/GPU:0'):
  history = model.fit(
    [train['user_id'], train['anime_id']], train['rating'],
    validation_data=([test['user_id'], test['anime_id']], test['rating']),
    epochs=20,
    batch_size=32,
    verbose=1
)

# Evaluate the model
test_loss, test_mae = model.evaluate([test['user_id'], test['anime_id']], test['rating'])
print(f"Test Loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")

# Save the model
model.save('anime_recommendation_model.h5')


Epoch 1/20
178223/178223 ━━━━━━━━━━━━━━━━━━━━ 481s 3ms/step - loss: 0.0161 - mae: 0.0967 - val_loss: 0.0139 - val_mae: 0.0896
Epoch 2/20
178223/178223 ━━━━━━━━━━━━━━━━━━━━ 477s 3ms/step - loss: 0.0134 - mae: 0.0870 - val_loss: 0.0135 - val_mae: 0.0873
Epoch 3/20
178223/178223 ━━━━━━━━━━━━━━━━━━━━ 477s 3ms/step - loss: 0.0128 - mae: 0.0845 - val_loss: 0.0133 - val_mae: 0.0859
Epoch 4/20
178223/178223 ━━━━━━━━━━━━━━━━━━━━ 516s 3ms/step - loss: 0.0122 - mae: 0.0820 - val_loss: 0.0131 - val_mae: 0.0860
Epoch 5/20
178223/178223 ━━━━━━━━━━━━━━━━━━━━ 478s 3ms/step - loss: 0.0116 - mae: 0.0798 - val_loss: 0.0130 - val_mae: 0.0851
Epoch 6/20
178223/178223 ━━━━━━━━━━━━━━━━━━━━ 520s 3ms/step - loss: 0.0112 - mae: 0.0778 - val_loss: 0.0129 - val_mae: 0.0852
Epoch 7/20
178223/178223 ━━━━━━━━━━━━━━━━━━━━ 476s 3ms/step - loss: 0.0108 - mae: 0.0759 - val_loss: 0.0129 - val_mae: 0.0846
Epoch 8/20
178223/178223 ━━━━━━━━━━━━━━━━━━━━ 512s 3ms/step - loss: 0.0104 - mae: 0.0743 - val_loss: 0.0129 - val_mae:

Test Loss (MSE): 0.0137
Test MAE: 0.0863


#Comparing Error of predicted values mean with Original vlaues mean

In [ ]:
global_mean = train['rating'].mean()
baseline_rmse = np.sqrt(np.mean((test['rating'] - global_mean)**2))
print(f"Baseline RMSE: {baseline_rmse:.4f}")


Baseline RMSE: 0.3761


#Recommending Animes for A single User

In [23]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Load the trained model (if you have already saved it)
from keras.losses import mean_squared_error as mse
from keras.models import load_model

# Load the model, explicitly registering the mse function
model = load_model('/content/Latest Model.h5', custom_objects={'mse': mse})


# Select a user for recommendations
selected_user_id = 3  # Example: choose the user_id for which you want recommendations

# Get all anime IDs (you can change the range if you have a smaller or larger dataset)
anime_ids = np.arange(num_animes)  # From 0 to num_animes-1, as anime_id is encoded from 0 to num_animes-1

# Predict ratings for all animes for the selected user
predicted_ratings = model.predict([np.full_like(anime_ids, selected_user_id), anime_ids])

# Create a DataFrame with predicted ratings and corresponding anime IDs
predicted_df = pd.DataFrame({
    'anime_id': anime_ids,
    'predicted_rating': predicted_ratings.flatten()
})

# Sort the DataFrame by predicted_rating in descending order (top rated animes first)
top_10_animes = predicted_df.sort_values(by='predicted_rating', ascending=False).head(10)

# Get the anime details for the top 10 animes
recommended_animes = anime_df[anime_df['anime_id'].isin(top_10_animes['anime_id'])]

# Display the recommended animes
print(f"Top 10 anime recommendations for User {selected_user_id}:")
print(recommended_animes[['anime_id', 'name', 'genre', 'type', 'episodes', 'rating']])


309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Top 10 anime recommendations for User 3:
      anime_id                                               name  \
905        540                  Tenchi Muyou! Ryououki 2nd Season   
917       1376                            Ojamajo Doremi Na-i-sho   
1905      6288  Mobile Suit Gundam 00 The Movie: A Wakening of...   
2486      1417                      Lupin III: Moeyo Zantetsuken!   
3077      6884                  Ookami to Koushinryou II Specials   
6093      6875                                           Iron Man   

                                              genre     type  episodes  rating  
905   Action, Comedy, Harem, Sci-Fi, Shounen, Space      OVA       6.0    7.78  
917                  Comedy, Fantasy, Magic, Shoujo      OVA      13.0    7.77  
1905                   Action, Mecha, Sci-Fi, Space    Movie       1.0    7.41  
2486             Action, Adventure, Comedy, Shounen  Special       1.0    7.25  
3077                         

#MAP@K AND R@K

In [ ]:
def calculate_metrics_at_k(model, ratings_df, anime_df, user_ids, k=10):
    """
    Calculate MAP@K and Recall@K for a list of users.

    Parameters:
        model: Trained recommendation model.
        ratings_df: DataFrame with user_id, anime_id, and actual ratings.
        anime_df: DataFrame with anime details (including anime_id).
        user_ids: List of user IDs to evaluate.
        k: Number of recommendations to consider (default: 10).

    Returns:
        map_k: Mean Average Precision at K.
        recall_k: Recall at K.
    """
    import numpy as np
    from sklearn.metrics import average_precision_score

    map_k = 0
    recall_k = 0
    relevant_users_count = 0

    for user_id in user_ids:
        # Get the actual relevant items for the user (rating >= threshold, e.g., 6)
        user_ratings = ratings_df[ratings_df['user_id'] == user_id]
        relevant_items = user_ratings[user_ratings['rating'] >= 0.6]['anime_id'].values  # Normalize threshold for relevant ratings

        if len(relevant_items) == 0:
            continue  # Skip users without relevant items

        relevant_users_count += 1

        # Predict ratings for all anime for this user
        # Ensure anime_ids are within the valid range (encoded values)
        anime_ids = anime_df['anime_id'].values
        predicted_ratings = model.predict([np.full_like(anime_ids, user_id), anime_ids], verbose=0).flatten()

        # Rank the anime by predicted rating
        top_k_anime_ids = anime_ids[np.argsort(-predicted_ratings)[:k]]

        # Calculate precision at K for this user
        is_relevant = np.isin(top_k_anime_ids, relevant_items).astype(int)
        average_precision = average_precision_score(is_relevant, np.arange(1, len(is_relevant) + 1))

        # Add to MAP@K
        map_k += average_precision

        # Calculate Recall@K
        recall_k += is_relevant.sum() / len(relevant_items)

    # Compute the average metrics
    if relevant_users_count > 0:
        map_k /= relevant_users_count
        recall_k /= relevant_users_count
    else:
        map_k = recall_k = 0  # Handle case where no relevant items are found

    return map_k, recall_k


# Example Usage
selected_users = ratings_df['user_id'].unique()[:10]  # Select the first 10 unique users
map_10, recall_10 = calculate_metrics_at_k(model, ratings_df, anime_df, selected_users, k=10)

print(f"MAP@10: {map_10:.4f}")
print(f"Recall@10: {recall_10:.4f}")


In [25]:
def hybrid_recommend_anime(user_id, interaction_matrix, pearson_correlation, k, anime_df, num_animes, model, top_n=5, alpha=0.5):
    """
    Hybrid recommendation system combining Neural Network and User-Based Collaborative Filtering (UBCF).
    """
    # Step 1: Predict ratings using the neural network model
    anime_ids = np.arange(num_animes)  # All anime IDs (from 0 to num_animes-1)
    predicted_ratings_nn = model.predict([np.full_like(anime_ids, user_id), anime_ids]).flatten()

    # Step 2: Predict ratings using User-Based Collaborative Filtering (UBCF)
    all_anime_ids = interaction_matrix.columns
    rated_anime_ids = interaction_matrix.loc[user_id][interaction_matrix.loc[user_id] > 0].index
    anime_to_predict = [anime_id for anime_id in all_anime_ids if anime_id not in rated_anime_ids]

    predictions_ubcf = []
    for anime_id in anime_to_predict:
        # Find similar users and their scores
        similar_users, scores = find_similar_users(k, user_id, pearson_correlation, interaction_matrix)
        predicted_rating_ubcf = prediction(similar_users, scores, interaction_matrix, user_id, anime_id)
        predictions_ubcf.append((anime_id, predicted_rating_ubcf))

    predictions_ubcf.sort(key=lambda x: x[1], reverse=True)
    top_anime_ids_ubcf = [anime_id for anime_id, _ in predictions_ubcf[:top_n]]

    # Step 3: Combine the NN and UBCF predictions
    combined_predictions = []

    # Map anime_id to the corresponding index in predicted_ratings_nn
    anime_id_to_index = {anime_id: idx for idx, anime_id in enumerate(anime_ids)}

    for anime_id in anime_to_predict:
        # Map anime_id to the correct index in predicted_ratings_nn
        nn_pred_idx = anime_id_to_index.get(anime_id, -1)  # Get the index, default to -1 if not found
        if nn_pred_idx != -1:
            nn_pred = predicted_ratings_nn[nn_pred_idx]
        else:
            nn_pred = 0  # Default to 0 if the anime_id is not found in predicted ratings

        # Find UBCF prediction (if available)
        ubcf_pred = next((rating for aid, rating in predictions_ubcf if aid == anime_id), 0)

        # Combine the predictions (you can adjust the alpha parameter to weight NN vs UBCF)
        combined_rating = alpha * nn_pred + (1 - alpha) * ubcf_pred
        combined_predictions.append((anime_id, combined_rating))

    # Sort by combined ratings and get the top N recommendations
    combined_predictions.sort(key=lambda x: x[1], reverse=True)
    top_anime_ids_combined = [anime_id for anime_id, _ in combined_predictions[:top_n]]

    # Step 4: Get the anime details for the top N recommendations
    recommended_anime = anime_df[anime_df['anime_id'].isin(top_anime_ids_combined)]

    return recommended_anime

# Example Usage
selected_user_id = 3
k = 5
top_n = 5
alpha = 0.6  # You can adjust this parameter for the hybrid approach

recommended_anime = hybrid_recommend_anime(selected_user_id, interaction_matrix, pearson_correlation, k, anime_df, num_animes, model, top_n, alpha)

# Display the recommended anime
print(f"Top {top_n} Hybrid Anime Recommendations for User {selected_user_id}:")
print(recommended_anime[['anime_id', 'name', 'genre', 'type', 'episodes', 'rating']])


309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Top 5 Hybrid Anime Recommendations for User 3:
      anime_id                             name  \
22           1                     Cowboy Bebop   
152          5  Cowboy Bebop: Tengoku no Tobira   
214          6                           Trigun   
2094         7               Witch Hunter Robin   

                                                  genre   type  episodes  \
22      Action, Adventure, Comedy, Drama, Sci-Fi, Space     TV      26.0   
152               Action, Drama, Mystery, Sci-Fi, Space  Movie       1.0   
214                              Action, Comedy, Sci-Fi     TV      26.0   
2094  Action, Drama, Magic, Mystery, Police, Superna...     TV      26.0   

      rating  
22      8.82  
152     8.40  
214     8.32  
2094    7.36  
